In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

In [43]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from datetime import datetime
import time
import random
import cv2
import pandas as pd
import numpy as np
import albumentations as A
import matplotlib.pyplot as plt
from albumentations.pytorch.transforms import ToTensorV2
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from glob import glob

from torchvision import transforms

SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

In [57]:
CONFIG = {"seed": 42,
          "epochs": 3,
          "img_size": 128,
          "model_name": "tf_efficientnet_b0",
          "num_classes": 15587,
          "train_batch_size": 32,
          "valid_batch_size": 64,
          "learning_rate": 1e-4,
          "scheduler": 'CosineAnnealingLR',
          "min_lr": 1e-6,
          "T_max": 500,
          "weight_decay": 1e-6,
          "n_fold": 5,
          "n_accumulate": 1,
          "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
          # ArcFace Hyperparameters
          "s": 30.0, 
          "m": 0.50,
          "ls_eps": 0.0,
          "easy_margin": False
          }

In [47]:
PATH = '/home/haohe/kaggle/whale'
train_data = PATH + '/data/img_256/train_images-256-256/train_images-256-256'
test_data = PATH + '/data/img_256/test_images-256-256/test_images-256-256'
df_train = pd.read_csv(PATH + '/data/raw_data/train.csv')
df_test = pd.read_csv(PATH + '/data/raw_data/sample_submission.csv')

In [48]:
df_train.head()

,image,species,individual_id
0,00021adfb725ed.jpg,melon_headed_whale,cadddb1636b9
1,000562241d384d.jpg,humpback_whale,1a71fbb72250
2,0007c33415ce37.jpg,false_killer_whale,60008f293a2b
3,0007d9bca26a99.jpg,bottlenose_dolphin,4b00fe572063
4,00087baf5cef7a.jpg,humpback_whale,8e5253662392


In [41]:
len(df_train.individual_id.unique())

15587

In [49]:
encoder = LabelEncoder()
df_train['label'] = encoder.fit_transform(df_train['individual_id'])

In [50]:
df_train.head()

,image,species,individual_id,label
0,00021adfb725ed.jpg,melon_headed_whale,cadddb1636b9,12348
1,000562241d384d.jpg,humpback_whale,1a71fbb72250,1636
2,0007c33415ce37.jpg,false_killer_whale,60008f293a2b,5842
3,0007d9bca26a99.jpg,bottlenose_dolphin,4b00fe572063,4551
4,00087baf5cef7a.jpg,humpback_whale,8e5253662392,8721


In [56]:
len(df_train.individual_id.unique())

15587

In [53]:
encoder.inverse_transform([12348])

array(['cadddb1636b9'], dtype=object)

In [54]:
class WhaleDataset(Dataset):
    def __init__(self,path,df,transforms=None):
        super().__init__()
        self.path = path
        self.df = df
        self.transforms = transforms
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,idx):
        fname, label = self.df.iloc[idx]['image'], self.df.iloc[idx]['label']
        image = cv2.imread(f'{self.path}/{fname}', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BRG2RGB)
        if self.transforms:
            aug = self.transforms(
                image = image,
            )
            image = aug['image']
        return image, torch.tensor(label, dtype=torch.long)

In [59]:
def get_train_transform():
    return A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Rotate(limit=30, p=0.5),
        A.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
        ToTensorV2()], p=1.)

def get_valid_transform():
    return A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
        ToTensorV2()], p=1.)